# Geração de instâncias

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
env = gp.Env()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br


In [2]:
from udgp import M1, M2, M4, M5, random_instance

# 12345 (átomo redundante)

N = 6
instance = random_instance(N, freq=True)

instance.view_input()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
m = M4(instance, n=4, env=env)
# m.Params.TimeLimit = 120
m.optimize()
m.instance.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
print(instance.dist)
print(instance.freq)
print(m.instance.dist)
print(m.instance.freq)

[1.5   2.449 2.45  2.872 3.464]
[5 3 2 4 1]
[1.5   2.449 2.45  2.872]
[1 2 1 1]


In [5]:
from scipy.spatial.distance import cdist, pdist

print(np.sort(pdist(instance.input_coords, "euclidean").round(2)))
print(np.sort(pdist(m.instance.coords, "euclidean").round(2)))

[1.5  1.5  1.5  1.5  1.5  2.45 2.45 2.45 2.45 2.45 2.87 2.87 2.87 2.87
 3.46]
[1.5  1.5  1.5  1.5  2.45 2.45 2.87 2.87 2.87 3.46]


In [6]:
from copy import deepcopy

previous_a = []
inst = deepcopy(instance)

while not inst.is_solved():
    print("tentativa")

    inst = deepcopy(instance)

    m = M4(inst, n=4, max_gap=1e-2, previous_a=previous_a, env=env)
    # m.Params.TimeLimit = 120
    m.optimize()
    inst = m.instance

    # if m.Status is not GRB.OPTIMAL:
    #     continue

    previous_a.append(list(m.a_ijk_values()))

    print(list(m.a_ijk_values()))

    while inst.fixed_n < inst.n:
        print("step")
        m = M4(inst, n=1, max_gap=1e-2, env=env)
        m.setParam('TimeLimit', 60)
        m.optimize()

        if m.Status is not GRB.OPTIMAL:
            break

        inst = m.instance

inst.view()


tentativa
[(0, 1, 0), (0, 2, 1), (0, 3, 2), (0, 4, 0), (1, 2, 3), (1, 3, 4), (1, 4, 0), (2, 3, 0), (2, 4, 3), (3, 4, 3)]
step
Modelo inviável.
tentativa
[(0, 1, 3), (0, 2, 2), (0, 3, 1), (0, 4, 1), (1, 2, 3), (1, 3, 3), (1, 4, 0), (2, 3, 1), (2, 4, 2), (3, 4, 4)]
step
Modelo inviável.
tentativa
[(0, 1, 3), (0, 2, 4), (0, 3, 3), (0, 4, 2), (1, 2, 0), (1, 3, 3), (1, 4, 0), (2, 3, 2), (2, 4, 0), (3, 4, 3)]
step
Modelo inviável.
tentativa
[(0, 1, 1), (0, 2, 0), (0, 3, 3), (0, 4, 2), (1, 2, 3), (1, 3, 0), (1, 4, 0), (2, 3, 1), (2, 4, 2), (3, 4, 0)]
step
Modelo inviável.
tentativa
[(0, 1, 0), (0, 2, 3), (0, 3, 2), (0, 4, 0), (1, 2, 1), (1, 3, 3), (1, 4, 1), (2, 3, 3), (2, 4, 2), (3, 4, 0)]
step
tentativa
[(0, 1, 0), (0, 2, 0), (0, 3, 1), (0, 4, 0), (1, 2, 2), (1, 3, 3), (1, 4, 2), (2, 3, 3), (2, 4, 1), (3, 4, 0)]
step
Modelo inviável.
tentativa
[(0, 1, 2), (0, 2, 3), (0, 3, 2), (0, 4, 0), (1, 2, 0), (1, 3, 0), (1, 4, 3), (2, 3, 3), (2, 4, 1), (3, 4, 4)]
step
Modelo inviável.
tentativa
[(0, 1

[(0, 1, 0), (0, 2, 1), (0, 3, 4), (0, 4, 3), (1, 2, 3), (1, 3, 3), (1, 4, 1), (2, 3, 2), (2, 4, 0), (3, 4, 0)]
step
tentativa
